## Exercise

### Objective

Build a conversational assistant that can interact with a Trello board to manage tasks using LangGraph and LangChain. Your assistant should be able to:

- List tasks from the "To Do" list.
- Add a task to the "To Do" list.
- Mark a task as "Done" if mentioned by the user.

You are building the LangGraph workflow (aside from the provided functions). Use [intro-langgraph-3.ipynb](./intro-langgraph-3.ipynb) as a reference.

You'll be able to see results in realtime in the Trello board shown in classroom. You can also check [here](https://trello.com/invite/b/682d94d7b1fdae6ee544b3fb/ATTI72b22a3043e7fe210ea59a12e8fd223fC7879CB6/agentic-ai-tasks).

In [1]:
from dotenv import load_dotenv
import os
import requests

load_dotenv()

True

In [15]:
# === TRELLO CONFIG ===
BOARD_ID = "li6KqjnE"
TODO_LIST_NAME = "To Do"
DONE_LIST_NAME = "Done"
TRELLO_KEY = os.getenv("TRELLO_KEY")
TRELLO_TOKEN = os.getenv("TRELLO_TOKEN")

def get_list_id(list_name):
    lists = requests.get(
        f"https://api.trello.com/1/boards/{BOARD_ID}/lists?key={TRELLO_KEY}&token={TRELLO_TOKEN}"
    ).json()
    return next((l["id"] for l in lists if l["name"] == list_name), None)

# === TOOLS ===
def fetch_trello_tasks() -> str:
    """Fetch tasks from Trello 'To Do' list."""
    list_id = get_list_id(TODO_LIST_NAME)
    cards = requests.get(
        f"https://api.trello.com/1/lists/{list_id}/cards?key={TRELLO_KEY}&token={TRELLO_TOKEN}"
    ).json()
    return "\n".join(card["name"] for card in cards) if cards else "No tasks found."

def mark_task_done(task_name: str) -> str:
    """Mark a Trello task as done."""
    list_id = get_list_id(DONE_LIST_NAME)
    all_cards = requests.get(
        f"https://api.trello.com/1/boards/{BOARD_ID}/cards?key={TRELLO_KEY}&token={TRELLO_TOKEN}"
    ).json()
    card = next((c for c in all_cards if c["name"].lower() in task_name.lower()), None)
    if card:
        requests.put(
            f"https://api.trello.com/1/cards/{card['id']}?idList={list_id}&key={TRELLO_KEY}&token={TRELLO_TOKEN}"
        )
        return f"Marked '{card['name']}' as done."
    return f"Task '{task_name}' not found."

def add_task(task_name: str) -> str:
    """Add a new task to Trello 'To Do' list."""
    list_id = get_list_id(TODO_LIST_NAME)
    requests.post(
        f"https://api.trello.com/1/cards?name={task_name}&idList={list_id}&key={TRELLO_KEY}&token={TRELLO_TOKEN}"
    )
    return f"Added task: '{task_name}'."

In [ ]:
## your code here
